입력데이터와 정답데이터(공식코드에서 meta 어쩌고 변수 이름을 쓰는 거)를 묶어서 한 파일로 저장.

입력데이터와 정답데이터에서 `Test_id`의 순서가 같지 않은데 이 둘의 순서 상관 없이 취급하도록 코드가 작성돼있어 결과적으로 정답 데이터가 잘못 된 거나 마찬가지임.

그래서 입력데이터 별 해당 정답(`Label` 컬럼)을 찾아서 같은 CSV 파일에 저장함.

In [1]:
import os
from typing import Callable
import pandas as pd
from mymodules.wkqehdtksl import Paths


In [2]:
PATH = Paths('open_v2')
CHUNK_SIZE = 10000

In [3]:
if os.path.exists(PATH.aggt_A):
    os.remove(PATH.aggt_A)
if os.path.exists(PATH.aggt_B):
    os.remove(PATH.aggt_B)

In [4]:
train_meta = pd.read_csv(PATH.train_meta, index_col='Test_id')
meta_A = train_meta[train_meta["Test"] == "A"].copy()
meta_B = train_meta[train_meta["Test"] == "B"].copy()

In [5]:
row_count = 0
reader = pd.read_csv(PATH.train_A, index_col='Test_id', chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    row_count += len(chunk)
    chunk['Label'] = chunk.index.map(meta_A['Label'])
    chunk.to_csv(PATH.aggt_A, mode='a', index='Test_id', header=(i == 0))

print(f'A 길이: {row_count}')

A 길이: 647241


In [6]:
row_count = 0
reader = pd.read_csv(PATH.train_B, index_col='Test_id', chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    chunk['Label'] = chunk.index.map(meta_B['Label'])
    chunk.to_csv(PATH.aggt_B, mode='a', index='Test_id', header=(i == 0))

print(f'B 길이: {row_count}')

B 길이: 0


검증

Label 1 개수 일치 확인.

agg, meta에서 Label 값 일치 확인.

In [7]:
count_1 = 0

reader = pd.read_csv(PATH.aggt_A, index_col='Test_id', chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    count_1 += chunk['Label'].sum()

count_1_from_meta = meta_A['Label'].sum()

print(f'[Label=1 개수] agg: {count_1} / meta: {count_1_from_meta}')
print('일치' if count_1 == count_1_from_meta else '불일치')

[Label=1 개수] agg: 14692 / meta: 14692
일치


In [8]:
count_1 = 0

reader = pd.read_csv(PATH.aggt_B, index_col='Test_id', chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    count_1 += chunk['Label'].sum()

count_1_from_meta = meta_B['Label'].sum()

print(f'[Label=1 개수] agg: {count_1} / meta: {count_1_from_meta}')
print('일치' if count_1 == count_1_from_meta else '불일치')

[Label=1 개수] agg: 12590 / meta: 12590
일치


In [9]:
reader = pd.read_csv(PATH.aggt_A, index_col='Test_id', usecols=['Test_id', 'Label'], chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    meta_join = meta_A.join(chunk, how='inner', lsuffix='_meta', rsuffix='_agg')
    result = meta_join[meta_join['Label_meta'] != meta_join['Label_agg']]
    if(len(result) > 0):
        print(result)
# 출력 없으면 통과.

In [10]:
reader = pd.read_csv(PATH.aggt_B, index_col='Test_id', usecols=['Test_id', 'Label'], chunksize=CHUNK_SIZE)
for i, chunk in enumerate(reader):
    meta_join = meta_B.join(chunk, how='inner', lsuffix='_meta', rsuffix='_agg')
    result = meta_join[meta_join['Label_meta'] != meta_join['Label_agg']]
    if(len(result) > 0):
        print(result)
# 출력 없으면 통과.